In [1]:
import os
import json
from llama_parse import LlamaParse
from dotenv import load_dotenv

load_dotenv()

os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv("Llama_API_KEY")


parser = LlamaParse(
    result_type="markdown",  
    num_workers=4,           
    language="ko",          
    verbose=True
)

def process_insurance_pdfs(file_paths):
    for path in file_paths:
        if not os.path.exists(path):
            print(f"파일을 찾을 수 없습니다: {path}")
            continue

        file_name = os.path.basename(path)

        # 파일명에서 확장자를 제외한 이름을 상품명으로 사용
        product_name = file_name.replace(".pdf", "")
        
        # 파일명에 '요약'이 들어있으면 summary, 없으면 policy(약관)로 분류
        doc_type = "summary" if "요약" in file_name else "policy"

        try:
            # LlamaParse로 PDF 파싱 실행
            documents = parser.load_data(path)
            
            # 페이지별 데이터를 담을 리스트
            structured_list = []
            
            for i, doc in enumerate(documents):
                page_num = i + 1
                
                # RAG 성능을 극대화하는 개별 청크 구조
                entry = {
                    "content": doc.text,  # 파싱된 마크다운 텍스트
                    "metadata": {
                        "source_file": file_name,
                        "product_name": product_name,
                        "doc_type": doc_type,
                        "page_number": page_num,
                        "chunk_id": f"{product_name}_{doc_type}_p{page_num}",

                    }
                }
                structured_list.append(entry)

            # [파일별 개별 저장] .json 형태
            output_filename = f"structured_{product_name}.json"
            with open(output_filename, "w", encoding="utf-8") as f:
                json.dump(structured_list, f, ensure_ascii=False, indent=4)
            
            print(f"저장 완료: {output_filename} (총 {len(structured_list)}페이지)")

        except Exception as e:
            print(f"{file_name} 처리 중 오류 발생: {e}")



# 직접 경로 지정 방식
pdf_path_1 = "data/무배당 현대해상 굿앤굿어린이종합보험Q(Hi2509).pdf"

# 처리할 대상 리스트
target_files = [pdf_path_1]

# 전체 프로세스 실행
if __name__ == "__main__":
    process_insurance_pdfs(target_files)

/Users/soyoung/BWLOVERS-RAG/BWLOVERS-RAG-AI/venv/lib/python3.14/site-packages/llama_cloud/core/jsonable_encoder.py:23: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  import pydantic.v1 as pydantic  # type: ignore


RuntimeError: no validator found for <class 'pydantic.v1.fields.UndefinedType'>, see `arbitrary_types_allowed` in Config

In [ ]:
import os
import time
import json
import pandas as pd
from datasets import Dataset
from google.colab import userdata
from dotenv import load_dotenv

# Ragas 및 LangChain 필수 임포트
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

embedding_model = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
router_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# JSON 파일 로드 및 메모리 Vector DB 생성
JSON_FILE_PATH = '/content/structured_KB 다이렉트 자녀보험(무배당)약관.json'

with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# JSON 데이터를 LangChain Document 객체로 변환
documents = []
for item in json_data:
    content = item.get("content", "")
    meta = item.get("metadata", {})
    
    new_doc = Document(
        page_content=content,
        metadata={
            "source": meta.get("source_file", "KB_Insurance"),
            "page": meta.get("page_number", "정보 없음")
        }
    )
    documents.append(new_doc)

# persist_directory를 지정하지 않아 메모리 상에만 존재하는 임시 DB 생성
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="kb_child_only"
)

# 검색기 정의 (상위 5개 조항 추출)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
print(f"✅ {len(documents)}개의 KB 약관 조항이 메모리 DB에 로드되었습니다.")

# 프롬프트 정의
base_system = """
[역할] 너는 보험 약관 전문 분석가다. 제공된 문맥(context)만 근거로 답하라.
[규칙]
1) {context}에 없는 내용은 '정보 없음'으로 답할 것.
2) 답변에는 반드시 해당 조항의 페이지(metadata의 page)를 포함할 것.
3) 불필요한 중복 보장을 피할 수 있는 대안을 제시할 것.
"""

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", base_system + "\n\n[질문 유형: 요약/검색]\n제공된 문맥:\n{context}"),
    ("human", "{input}")
])

recommend_prompt = ChatPromptTemplate.from_messages([
    ("system", base_system + "\n\n[질문 유형: 추천]\n제공된 문맥:\n{context}"),
    ("human", "{input}")
])

# RAG 체인 구축
summary_chain = create_retrieval_chain(
    retriever,
    create_stuff_documents_chain(llm, summary_prompt)
)
recommend_chain = create_retrieval_chain(
    retriever,
    create_stuff_documents_chain(llm, recommend_prompt)
)

def classify_query(query: str) -> str:
    # 간단한 라우팅 로직
    if "추천" in query or "맞는" in query:
        return "recommend"
    return "summary_search"

def ask_question_for_ragas(query: str):
    qtype = classify_query(query)
    if qtype == "recommend":
        response = recommend_chain.invoke({"input": query})
    else:
        response = summary_chain.invoke({"input": query})
    
    return {
        "question": query,
        "answer": response["answer"],
        "contexts": [doc.page_content for doc in response["context"]]
    }

# 질문 실행 및 Ragas 평가
test_questions = [
    "임신 10주차 산모를 위한 추천 특약과 그 이유를 약관 근거(페이지 포함)를 들어 설명해줘.",
    "이 보험 약관에서 아이가 입원했을 때 보장받을 수 있는 조항들을 요약해줘.",
    "아동의 암 진단과 관련된 조항을 찾아줘."
]

rag_data = []
for q in test_questions:
    print(f"질문 처리 중: {q[:30]}...")
    rag_data.append(ask_question_for_ragas(q))

# Ragas 데이터셋 변환 및 평가
df = pd.DataFrame(rag_data)
ragas_dataset = Dataset.from_pandas(df)

results = evaluate(
    dataset=ragas_dataset,
    metrics=[faithfulness, answer_relevancy],
    llm=llm
)

# 결과 출력
print("\n📊 Ragas 평가 결과 (KB 약관 전용)")
print(results.to_pandas()[['faithfulness', 'answer_relevancy']].mean())

In [ ]:
df_results = results.to_pandas()

q_col = 'question' if 'question' in df_results.columns else 'user_input'
a_col = 'answer' if 'answer' in df_results.columns else 'response'
c_col = 'contexts' if 'contexts' in df_results.columns else 'retrieved_contexts'

print("\n" + "="*50)
print("모든 질문에 대한 RAG 답변 및 평가 결과")
print("="*50)

for i, row in df_results.iterrows():
    print(f"\n[질문 {i+1}]")
    print(f"❓ {row[q_col]}")  
    print("-" * 30)
    print(f"[LLM의 답변]\n{row[a_col]}")
    print("-" * 30)
    
    f_score = row.get('faithfulness', 0)
    r_score = row.get('answer_relevancy', 0)
    print(f"📊 평가 점수: 충실도 {f_score:.2f} | 관련성 {r_score:.2f}")
    
    print(f"\n[참조 문맥 일부]")
    for j, ctx in enumerate(row[c_col]):
      print(f"문맥 {j+1}: {ctx[:150]}...")
    
    print("\n" + "="*50)